In [1]:
# Import all required libraries
import pandas as pd
import numpy as np
import tweepy
import json
import requests
import os

# Gather:

There are three datasets to be gathered from various sources.

1. twitter-archive-enhanced.csv contains basic tweet information.
2. Collect additional information about the above tweets from twitter using twitter api. Read tweet_id, retweet_count, favorite_count from the tweet status into a data frame.
3. image_predictions.tsv is a predictions file that contains dog breed prediction. The file is hosted  at https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv


In [2]:
## Load the first dataset from twitter-archive-enhanced.csv
tweets = pd.read_csv("twitter-archive-enhanced.csv")

In [3]:
### Look up tweets data frame.
tweets.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


In [4]:
## Collect 
consumer_key = 'nTPtvUykjiJ441MuyPF6tZ5g0'
consumer_secret = 'czyVQtnt2K73RIZ8dSthnxhaINPVbWfPnvpfHJ1lKMTSYUQmeA'
access_token = '3830378115-uB6bkozFAxq2UIJ9pgSmccfUOmJMO2AdabsyXcF'
access_secret = '4CNZFZdWYcI1srNE4QOhWszeAU5gI7c16HrY0z5u8YgLi'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

# Set the wait_on_rate_limit and wait_on_rate_limit_notify to True
# wait_on_rate_limit – 
#    Whether or not to automatically wait for rate limits to replenish
# wait_on_rate_limit_notify – 
#    Whether or not to print a notification when Tweepy is waiting 
#    for rate limits to replenish
api = tweepy.API(
    auth, 
    wait_on_rate_limit=True, 
    wait_on_rate_limit_notify=True)

In [5]:
# Read additional information about tweets from twitter if it is not already present
# Proceed with accessing information using tweepy if tweet_json.txt is not present.
tweetInfoFileName = "tweet_json.txt"
import os
if not os.path.exists(tweetInfoFileName): 
    tweetData = dict()
    for tweet_id in tweets.tweet_id:
        try:
            tweet = api.get_status(tweet_id, tweet_mode='extended')
            tweetData[tweet_id] = tweet._json
        except:
            print("No status found with that ID:{}".format(tweet_id))

    # Store all json into a file.
    with open(tweetInfoFileName, 'w') as fp:
        json.dump(list(tweetData.values()),fp)
else:
    print("The tweet information is already downloaded and saved.")
    print("Proceed with loading the data from {} file".\
          format(tweetInfoFileName))

The tweet information is already downloaded and saved.
Proceed with loading the data from tweet_json.txt file


In [6]:
# Read all tweets from the json dump.
tweetList = list()
with open(tweetInfoFileName, 'r') as fp:
    tweetList = json.load(fp)

In [7]:
# Load into a data frame
# Read tweet_id, retweet_count and favourites_count into a data frame.
tweet_moreInfo = pd.DataFrame(
    columns=['tweet_id','retweet_count','favorite_count'])
for tweet in tweetList:
    tweet_id = tweet.get('id') if 'id' in tweet else None
    retweet_count = tweet.get('retweet_count') \
    if 'retweet_count' in tweet else None
    favorite_count = tweet.get('favorite_count') \
    if 'favorite_count' in tweet else None
    
    if tweet_id:
        tweet_moreInfo = tweet_moreInfo.append(
            {'tweet_id':tweet_id, 
             'retweet_count':retweet_count, 
             'favorite_count':favorite_count}, 
            ignore_index=True)
    
# Test for gathering the dataset
tweet_moreInfo.head()

,tweet_id,retweet_count,favorite_count
0,892420643555336193,8655,38999
1,892177421306343426,6355,33391
2,891815181378084864,4222,25176
3,891689557279858688,8767,42351
4,891327558926688256,9533,40515


In [8]:
# Now load the third data set.
# Download it from the url
url = "https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv"
response = requests.get(url)

# Save it in a file
with open("image_predictions.tsv", 'w') as fp:
    fp.write(response.content.decode())

# Load it dataset in a data frame
tweets_prediction = pd.read_csv("image_predictions.tsv", sep='\t')

# Test for gathering the dataset
tweets_prediction.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


# Assess
In this section, assess all three data sets for quality issues and tidiness issues. The various issues observed during assessment are listed below.

## Quality issues¶
### tweets

* in_reply_to_status_id, in_reply_to_user_id, retweeted_status_id and retweeted_status_user_id are float64 instead of int64.  
* time_stamp and retweeted_status_timestamp are object instead of type date
* For tweet_id 835246439529840640, rating is 13/10 instead of 960/0. 
* For tweet_id 835152434251116546, rating is 0/10. Although the rating is not wrong as per the tweet text, no dog deserves a 0. This could be a typing error from the user. This could probably be quality issue. 
* The rating system doesn't have any standard. Denominator isn't necessarily 10 and numerator isn't necessarily greater than 10. Also, these ratings doesn't seem to be normalized and/or compared.
* The field name is an object and 745 entries are filled with None instead of NaN. This leads to misconception that all tweets have names. 
* Some names are extracted wrongly and contains articles such as 'a', 'an' and 'the' for names. 
* For tweet_id 670361874861563904, name = a instead of Cherokee.
* For tweet_id 670303360680108032, name = a instead of Hemry. Is it Henry or Hemry? Could it be typing error?
* The dog stages are filled with None and the field is of type object.Since, the counts are 2345 for all stages, it is misleading to believe that all stages are valid for all dogs.

### tweet_moreInfo

* More information is obtained from twitter for 2345 tweets out of 2356. 11 tweets were not present or not accessible anymore.
* All fields, tweet_id, retweet_count, and favorite_count are of type object instead of int.

### tweets_prediction

* Predictions are obtained only for 2075 entries out of 2356. There are missing entries.
* Need to check if there are any None for p1, p2, p3 predictions since these are of type object.

## Tidiness issues

* The first dataset "tweets" contains two sets of information, namely, static information about tweet and dog information extrated from the text.
* One characteristic, dog stage, is separated into multiple columns one each for each stage of dog. 
* tweets_moreInfo should be part of tweets.


## Assess tweets

In [9]:
# Checking for tweets quality
tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [10]:
tweets.describe(include='all')

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
count,2.356000e+03,7.800000e+01,7.800000e+01,2356,2356,2356,1.810000e+02,1.810000e+02,181,2297,2356.000000,2356.000000,2356,2356,2356,2356,2356
unique,NaN,NaN,NaN,2356,4,2356,NaN,NaN,181,2218,NaN,NaN,957,2,2,2,2
top,NaN,NaN,NaN,2016-01-11 01:25:58 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Daniel. He's a neat pup. Exotic af. Cu...,NaN,NaN,2016-09-25 00:06:08 +0000,https://www.gofundme.com/help-lorenzo-beat-can...,NaN,NaN,None,None,None,None,None
freq,NaN,NaN,NaN,1,2221,1,NaN,NaN,1,2,NaN,NaN,745,2259,2346,2099,2326
mean,7.427716e+17,7.455079e+17,2.014171e+16,NaN,NaN,NaN,7.720400e+17,1.241698e+16,NaN,NaN,13.126486,10.455433,NaN,NaN,NaN,NaN,NaN
std,6.856705e+16,7.582492e+16,1.252797e+17,NaN,NaN,NaN,6.236928e+16,9.599254e+16,NaN,NaN,45.876648,6.745237,NaN,NaN,NaN,NaN,NaN
min,6.660209e+17,6.658147e+17,1.185634e+07,NaN,NaN,NaN,6.661041e+17,7.832140e+05,NaN,NaN,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN
25%,6.783989e+17,6.757419e+17,3.086374e+08,NaN,NaN,NaN,7.186315e+17,4.196984e+09,NaN,NaN,10.000000,10.000000,NaN,NaN,NaN,NaN,NaN
50%,7.196279e+17,7.038708e+17,4.196984e+09,NaN,NaN,NaN,7.804657e+17,4.196984e+09,NaN,NaN,11.000000,10.000000,NaN,NaN,NaN,NaN,NaN
75%,7.993373e+17,8.257804e+17,4.196984e+09,NaN,NaN,NaN,8.203146e+17,4.196984e+09,NaN,NaN,12.000000,10.000000,NaN,NaN,NaN,NaN,NaN


In [11]:
# in_reply_to_status_id and in_reply_to_user_id are float instead of int. 
# Check values if there are any wrong entries since ids should not be float.
tweets.in_reply_to_status_id.value_counts()

6.671522e+17    2
8.562860e+17    1
8.131273e+17    1
6.754971e+17    1
6.827884e+17    1
8.265984e+17    1
6.780211e+17    1
6.689207e+17    1
6.658147e+17    1
6.737159e+17    1
7.590995e+17    1
8.862664e+17    1
7.384119e+17    1
7.727430e+17    1
7.468859e+17    1
8.634256e+17    1
6.693544e+17    1
6.914169e+17    1
6.920419e+17    1
6.753494e+17    1
7.291135e+17    1
8.406983e+17    1
6.747400e+17    1
7.501805e+17    1
6.744689e+17    1
7.638652e+17    1
6.747934e+17    1
8.503288e+17    1
6.747522e+17    1
8.816070e+17    1
               ..
8.380855e+17    1
8.211526e+17    1
8.558616e+17    1
8.558585e+17    1
7.032559e+17    1
6.678065e+17    1
8.018543e+17    1
7.667118e+17    1
6.855479e+17    1
6.717299e+17    1
6.715610e+17    1
6.758457e+17    1
6.924173e+17    1
7.476487e+17    1
8.381455e+17    1
6.903413e+17    1
8.476062e+17    1
8.352460e+17    1
6.813394e+17    1
8.795538e+17    1
6.860340e+17    1
8.571567e+17    1
6.765883e+17    1
7.044857e+17    1
8.707262e+

In [12]:
# Look up some sample entries to feel the values for each column.
tweets.sample(5)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
54,881666595344535552,NaN,NaN,2017-07-03 00:11:11 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Gary. He couldn't miss this puppertuni...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/881666595...,13,10,Gary,None,None,None,None
378,827933404142436356,NaN,NaN,2017-02-04 17:34:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Kona. Yesterday she stopped by the dep...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/827933404...,12,10,Kona,None,None,pupper,None
2197,668826086256599040,NaN,NaN,2015-11-23 16:18:59 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Banditt. He is a brown LaBeouf retriev...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/668826086...,10,10,Banditt,None,None,None,None
2249,667861340749471744,NaN,NaN,2015-11-21 00:25:26 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a Shotokon Macadamia mix named Cheryl....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/667861340...,9,10,a,None,None,None,None
740,780543529827336192,NaN,NaN,2016-09-26 23:04:13 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here's a perturbed super floof. 12/10 would sn...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/780543529...,12,10,None,None,None,None,None


Couple of things observed from above sample.
* rating_numerator can be lesser than 10 or it is wrongly extracted.
* Names could be None. But they are still considered as objects. So, we don't know exactly how many entries are available for name.
* Same for different stages of dogs.


In [13]:
# Lets check how many entires have rating_numerator < 10
len(tweets[tweets.rating_numerator < 10])

440

In [14]:
# Set the display settings to show all text.
pd.set_option('display.max_colwidth', -1)

# Verify ratings from text.
tweets[tweets.rating_numerator < 10].sample(5)[
    ['tweet_id','text', 'rating_numerator', 'rating_denominator']]

,tweet_id,text,rating_numerator,rating_denominator
1739,679511351870550016,Say hello to William. He makes fun of others because he's terrified of his own deep-seated insecurities. 7/10 https://t.co/bwuV6FlRxr,7,10
1604,685906723014619143,This is Olive. He's stuck in a sleeve. 9/10 damn it Olive https://t.co/NnLjg6BgyF,9,10
1559,688547210804498433,This is Frönq. He got caught stealing a waffle. Damn it Frönq. 9/10 https://t.co/7ycWCUrjmZ,9,10
1699,680970795137544192,I thought I made this very clear. We only rate dogs. Stop sending other things like this shark. Thank you... 9/10 https://t.co/CXSJZ4Stk3,9,10
2181,668994913074286592,Two gorgeous pups here. Both have cute fake horns(adorable). Barn in the back looks on fire. 5/10 would pet rly well https://t.co/w5oYFXi0uh,5,10


In [15]:
# Lets get the frequency of various names
tweets.name.value_counts()

None        745
a           55 
Charlie     12 
Cooper      11 
Oliver      11 
Lucy        11 
Lola        10 
Penny       10 
Tucker      10 
Bo          9  
Winston     9  
Sadie       8  
the         8  
Daisy       7  
Buddy       7  
Bailey      7  
an          7  
Toby        7  
Stanley     6  
Bella       6  
Milo        6  
Dave        6  
Rusty       6  
Leo         6  
Oscar       6  
Koda        6  
Scout       6  
Jack        6  
Jax         6  
Bentley     5  
           ..  
Jo          1  
Joshwa      1  
Charl       1  
Robin       1  
Kaiya       1  
Damon       1  
Cedrick     1  
Rontu       1  
Snoopy      1  
Cheesy      1  
Cheryl      1  
Tyrus       1  
Norman      1  
Edgar       1  
Lucia       1  
Bobbay      1  
Marlee      1  
Tayzie      1  
Eugene      1  
Pumpkin     1  
Trip        1  
Spencer     1  
Arnold      1  
Tedders     1  
Humphrey    1  
Ito         1  
Snoop       1  
Laika       1  
Fabio       1  
Ralphy      1  
Name: name, Length: 957,

Observations:
* Some names are 'a', 'an', 'the'. 
* 745 entries are None.

In [16]:
# Lets check if tweets contains dog names at all for above observations.
tweets[tweets.name == 'a'].sample(5)[['tweet_id','text','name']]

,tweet_id,text,name
1853,675706639471788032,This is a Sizzlin Menorah spaniel from Brooklyn named Wylie. Lovable eyes. Chiller as hell. 10/10 and I'm out.. poof https://t.co/7E0AiJXPmI,a
1002,747885874273214464,This is a mighty rare blue-tailed hammer sherk. Human almost lost a limb trying to take these. Be careful guys. 8/10 https://t.co/TGenMeXreW,a
1854,675534494439489536,Seriously guys?! Only send in dogs. I only rate dogs. This is a baby black bear... 11/10 https://t.co/H7kpabTfLj,a
2116,670427002554466305,This is a Deciduous Trimester mix named Spork. Only 1 ear works. No seat belt. Incredibly reckless. 9/10 still cute https://t.co/CtuJoLHiDo,a
2218,668507509523615744,This is a Birmingham Quagmire named Chuk. Loves to relax and watch the game while sippin on that iced mocha. 10/10 https://t.co/HvNg9JWxFt,a


In [17]:
# Now lets go to dog stages and various values.
tweets.puppo.value_counts()

None     2326
puppo    30  
Name: puppo, dtype: int64

## Assess tweet_moreInfo

In [18]:
# Checking for tweets_moreInfo quality
tweet_moreInfo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2345 entries, 0 to 2344
Data columns (total 3 columns):
tweet_id          2345 non-null object
retweet_count     2345 non-null object
favorite_count    2345 non-null object
dtypes: object(3)
memory usage: 55.0+ KB


Observations:
* All columns are of type object.

In [19]:
# Lets check if counts are integers or not.
tweet_moreInfo.retweet_count.value_counts()

2637     6
1315     6
1283     5
600      5
94       5
514      4
246      4
594      4
884      4
747      4
567      4
45       4
252      4
560      4
235      4
1184     4
1007     4
100      3
2183     3
287      3
133      3
861      3
81       3
6387     3
131      3
532      3
2172     3
891      3
86       3
2914     3
        ..
1880     1
1891     1
1903     1
2238     1
6055     1
1916     1
1925     1
1842     1
1840     1
1837     1
1798     1
4349     1
1784     1
1785     1
1786     1
18171    1
645      1
1796     1
1797     1
4802     1
5932     1
1809     1
14101    1
5912     1
1823     1
106      1
1830     1
1831     1
1834     1
0        1
Name: retweet_count, Length: 1705, dtype: int64

In [20]:
# Lets check the distribution of counts.
tweet_moreInfo.describe(include='all')

,tweet_id,retweet_count,favorite_count
count,2345,2345,2345
unique,2345,1705,2008
top,667495797102141441,2637,0
freq,1,6,170


Observations:
* Since the columns are of type objects, the describe function is showing description of categorical data instead of numerical data.

In [21]:
tweet_moreInfo.sample(5)

,tweet_id,retweet_count,favorite_count
237,846153765933735936,9946,33794
451,817777686764523521,3011,11695
290,837110210464448512,2662,17209
1696,680801747103793152,911,2547
1425,697255105972801536,1284,3254


## Assess tweets_prediction

In [22]:
# Checking for tweets_prediction quality
tweets_prediction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [23]:
tweets_prediction.describe(include='all')

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
count,2.075000e+03,2075,2075.000000,2075,2075.000000,2075,2075,2.075000e+03,2075,2075,2.075000e+03,2075
unique,NaN,2009,NaN,378,NaN,2,405,NaN,2,408,NaN,2
top,NaN,https://pbs.twimg.com/media/CWyD2HGUYAQ1Xa7.jpg,NaN,golden_retriever,NaN,True,Labrador_retriever,NaN,True,Labrador_retriever,NaN,True
freq,NaN,2,NaN,150,NaN,1532,104,NaN,1553,79,NaN,1499
mean,7.384514e+17,NaN,1.203855,NaN,0.594548,NaN,NaN,1.345886e-01,NaN,NaN,6.032417e-02,NaN
std,6.785203e+16,NaN,0.561875,NaN,0.271174,NaN,NaN,1.006657e-01,NaN,NaN,5.090593e-02,NaN
min,6.660209e+17,NaN,1.000000,NaN,0.044333,NaN,NaN,1.011300e-08,NaN,NaN,1.740170e-10,NaN
25%,6.764835e+17,NaN,1.000000,NaN,0.364412,NaN,NaN,5.388625e-02,NaN,NaN,1.622240e-02,NaN
50%,7.119988e+17,NaN,1.000000,NaN,0.588230,NaN,NaN,1.181810e-01,NaN,NaN,4.944380e-02,NaN
75%,7.932034e+17,NaN,1.000000,NaN,0.843855,NaN,NaN,1.955655e-01,NaN,NaN,9.180755e-02,NaN


In [24]:
tweets_prediction.sample(5)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
346,672466075045466113,https://pbs.twimg.com/media/CVUUU_EWoAAxABV.jpg,1,cocker_spaniel,0.150424,True,toy_poodle,0.088605,True,Welsh_springer_spaniel,0.072014,True
1702,817171292965273600,https://pbs.twimg.com/media/C1cs8uAWgAEwbXc.jpg,1,golden_retriever,0.295483,True,Irish_setter,0.144431,True,Chesapeake_Bay_retriever,0.077879,True
1385,766008592277377025,https://pbs.twimg.com/media/CqFouXOXYAAYpzG.jpg,1,Welsh_springer_spaniel,0.728153,True,basset,0.103842,True,Brittany_spaniel,0.062414,True
2012,879050749262655488,https://pbs.twimg.com/media/DDMD_phXoAQ1qf0.jpg,1,tabby,0.311861,False,window_screen,0.169123,False,Egyptian_cat,0.132932,False
1953,863907417377173506,https://pbs.twimg.com/media/C_03NPeUQAAgrMl.jpg,1,marmot,0.358828,False,meerkat,0.174703,False,weasel,0.123485,False


# Clean
In this section, clean all the quality issues and tidiness issues found above. 

Before cleaning, however, copy all datasets and then modify the copies and save them. 


In [25]:
tweets_clean = tweets.copy()
tweets_moreInfo_clean = tweet_moreInfo.copy()
tweets_prediction_clean = tweets_prediction.copy()

## Cleaning of the above issues is done in the following order.
1. Missing information is cleaned first.
2. Tidiness issues are cleaned next.
3. Other quality issues are cleaned last.

## Issue 1:
### Definition:
Missing rows for tweet_moreInfo and tweets_prediction. 

### Cleaning:
For tweet_moreInfo, the entries are missing since the tweets doesn't exist anymore, otherwise the api would have collected them. As such, this missing information cannot be collected.

For tweets_prediction, the dog breed was predicted by a Machine Learning algorithm. The missing entries in this dataset can be filled by the following ways. 
* May be the algorithm couldn't predict the breed. In this case, we can try using different algorithm. However, since we don't have one, we can rest this.
* We can manually look at the dog images of remaining tweets and label them for p1. However, this task requires knowledge of dog breeds. Also, it gets complicated for mix breeds. Since, we don't have this expertise, we can ignore this idea as well. 

### Testing:
Since this issue couldn't be cleaned, there is no testing to be done.

## Issue 2:
### Definition:
The first dataset "tweets" contains two sets of information, namely, static information about tweet and dog information extrated from the text. Separate both of them into tweets_info and dog_info with tweet_id as common unique identity to join both tables.

### Cleaning:

In [26]:
tweets_clean.columns.values

array(['tweet_id', 'in_reply_to_status_id', 'in_reply_to_user_id',
       'timestamp', 'source', 'text', 'retweeted_status_id',
       'retweeted_status_user_id', 'retweeted_status_timestamp',
       'expanded_urls', 'rating_numerator', 'rating_denominator', 'name',
       'doggo', 'floofer', 'pupper', 'puppo'], dtype=object)

In [27]:
tweets_info = tweets_clean[
    ['tweet_id', 'in_reply_to_status_id', 'in_reply_to_user_id',
       'timestamp', 'source', 'text', 'retweeted_status_id',
       'retweeted_status_user_id', 'retweeted_status_timestamp',
       'expanded_urls']]
dogs_info = tweets_clean[
    ['tweet_id', 'name', 'rating_numerator', 'rating_denominator', 
       'doggo', 'floofer', 'pupper', 'puppo']]
del tweets_clean

### Testing:

In [28]:
tweets_info.info()
dogs_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 10 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
dtypes: float64(4), int64(1), object(5)
memory usage: 184.1+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 8 columns):
tweet_id              2356 non-null int64
name                  2356 non-null object
rating_numerator      2356 non-null int64
rating_denominator    2356 non-null int64
doggo                 2356 non-null object
floof

In [29]:
tweets_info.head()
dogs_info.head()

,tweet_id,name,rating_numerator,rating_denominator,doggo,floofer,pupper,puppo
0,892420643555336193,Phineas,13,10,None,None,None,None
1,892177421306343426,Tilly,13,10,None,None,None,None
2,891815181378084864,Archie,12,10,None,None,None,None
3,891689557279858688,Darla,13,10,None,None,None,None
4,891327558926688256,Franklin,12,10,None,None,None,None


## Issue 3:
### Definition:
One characteristic, dog stage, is separated into multiple columns one each for each stage of dog. Convert them into just one column called stage.

### Cleaning: 

In [30]:
dogs_info.columns.values

array(['tweet_id', 'name', 'rating_numerator', 'rating_denominator',
       'doggo', 'floofer', 'pupper', 'puppo'], dtype=object)

In [31]:
# Melt to convert auralin and novodra into treatment
import numpy as np
dogs_info.loc[:,'stage'] = np.nan
dogs_info.head()


,tweet_id,name,rating_numerator,rating_denominator,doggo,floofer,pupper,puppo,stage
0,892420643555336193,Phineas,13,10,None,None,None,None,NaN
1,892177421306343426,Tilly,13,10,None,None,None,None,NaN
2,891815181378084864,Archie,12,10,None,None,None,None,NaN
3,891689557279858688,Darla,13,10,None,None,None,None,NaN
4,891327558926688256,Franklin,12,10,None,None,None,None,NaN


In [32]:
for i, row in dogs_info.iterrows():
    if row['doggo'] == 'doggo': dogs_info.loc[i,'stage'] = 'doggo'
    elif row['floofer'] == 'floofer': dogs_info.loc[i,'stage'] = 'floofer'
    elif row['pupper'] == 'pupper': dogs_info.loc[i,'stage'] = 'pupper'
    elif row['puppo'] == 'puppo': dogs_info.loc[i,'stage'] = 'puppo'
    
dogs_info = dogs_info.drop(['doggo','floofer','pupper','puppo'], axis=1)

In [33]:
dogs_info.head()

,tweet_id,name,rating_numerator,rating_denominator,stage
0,892420643555336193,Phineas,13,10,NaN
1,892177421306343426,Tilly,13,10,NaN
2,891815181378084864,Archie,12,10,NaN
3,891689557279858688,Darla,13,10,NaN
4,891327558926688256,Franklin,12,10,NaN


### Testing:

In [34]:
dogs_info.stage.value_counts()

pupper     245
doggo      97 
puppo      29 
floofer    9  
Name: stage, dtype: int64

## Issue 4: 
### Definition:
The data set tweets_moreInfo should be part of tweets_info. Merge two tables on tweet_id.

### Cleaning:

In [35]:
# Merge tweet_moreInfo with tweetInfo
tweets_info = pd.merge(
    tweets_info, tweets_moreInfo_clean, on=['tweet_id'])

### Testing:

In [36]:
tweets_info.columns.values

array(['tweet_id', 'in_reply_to_status_id', 'in_reply_to_user_id',
       'timestamp', 'source', 'text', 'retweeted_status_id',
       'retweeted_status_user_id', 'retweeted_status_timestamp',
       'expanded_urls', 'retweet_count', 'favorite_count'], dtype=object)

In [37]:
len(tweets_info)

2345

In [38]:
tweets_info.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,retweet_count,favorite_count
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643555336193/photo/1,8655,38999
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https://t.co/0Xxu71qeIV",NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421306343426/photo/1,6355,33391
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know when one may strike. 12/10 https://t.co/wUnZnhtVJB,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181378084864/photo/1,4222,25176
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Darla. She commenced a snooze mid meal. 13/10 happens to the best of us https://t.co/tD36da7qLQ,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557279858688/photo/1,8767,42351
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Franklin. He would like you to stop calling him ""cute."" He is a very fierce shark and should be respected as such. 12/10 #BarkWeek https://t.co/AtUZn91f7f",NaN,NaN,NaN,"https://twitter.com/dog_rates/status/891327558926688256/photo/1,https://twitter.com/dog_rates/status/891327558926688256/photo/1",9533,40515


In [39]:
tweets_prediction_clean.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


## Issue 5:
### Define
Various columns in tweets are of float data type instead of int. The list of columns are mentioned below.
* in_reply_to_status_id
* in_reply_to_user_id
* retweeted_status_id
* retweeted_status_user_id   

### Clean
While cleaning to change the data type it was realized that int data type doesn't support NaN values. That is the reason the datatype was recognized as float. So, this will be kept as it is.

### Test
Since, no changes were made, no testing will be done.




## Issue 6:
### Definition:
Convert timestamp and retweeted_status_timestamp from datatype object to  date.

Also, convert retweet_count and favorite_count from object to int.

### Cleaning:

In [40]:
tweets_info.columns.values
tweets_info['timestamp'] = pd.to_datetime(tweets_info['timestamp'])
tweets_info['retweeted_status_timestamp'] = pd.to_datetime(
    tweets_info['retweeted_status_timestamp'])

tweets_info['retweet_count'] = tweets_info['retweet_count'].astype(int)
tweets_info['favorite_count'] = tweets_info['favorite_count'].astype(int)

### Testing:

In [41]:
tweets_info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2345 entries, 0 to 2344
Data columns (total 12 columns):
tweet_id                      2345 non-null object
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2345 non-null datetime64[ns]
source                        2345 non-null object
text                          2345 non-null object
retweeted_status_id           170 non-null float64
retweeted_status_user_id      170 non-null float64
retweeted_status_timestamp    170 non-null datetime64[ns]
expanded_urls                 2286 non-null object
retweet_count                 2345 non-null int64
favorite_count                2345 non-null int64
dtypes: datetime64[ns](2), float64(4), int64(2), object(4)
memory usage: 238.2+ KB


## Issue 7:
### Definition:
* For tweet_id 835246439529840640, set rating as 13/10 instead of 960/0. 
* For tweet_id 835152434251116546, rating is 0/10. Leave this as it is since we can't confirm the typing error in a tweet. 
* For tweet_id 670361874861563904, set name = Cherokee.
* For tweet_id 670303360680108032, set name = Hemry.

### Cleaning:


In [42]:
tweets_info[tweets_info.tweet_id == 835246439529840640]
dogs_info[dogs_info.tweet_id == 670361874861563904]
dogs_info[dogs_info.tweet_id == 670303360680108032]


,tweet_id,name,rating_numerator,rating_denominator,stage
2128,670303360680108032,a,9,10,NaN


In [43]:
tweets_info.loc[305,'rate_numerator'] = 13
tweets_info.loc[305,'rate_denominator'] = 10
dogs_info.loc[2125,'name'] = 'Cherokee'
dogs_info.loc[2128,'name'] = 'Hemry'


### Testing:

In [44]:
tweets_info[tweets_info.tweet_id == 835246439529840640]
dogs_info[dogs_info.tweet_id == 670361874861563904]
dogs_info[dogs_info.tweet_id == 670303360680108032]

,tweet_id,name,rating_numerator,rating_denominator,stage
2128,670303360680108032,Hemry,9,10,NaN


## Issue 8:
### Definition:
Dog names and ratings are wrongly extracted. 

For names, we can either do it manually or using NLP. However, manual updating is not feasible and NLP based entity name extraction is out of scope. As such name will not be updated.

For ratings, it looks like the wrong extraction is only in case there are multiple ratings in the tweet. Currently the ratings are taken from first pair but it should be taken from last pair. However, this is not certain. Since, we have already updated couple of entries we visually assessed, we'll retain the rating extraction as it is.

### Cleaning:
No Cleaning

### Testing:
No testing is required.

## Issue 9:
### Definition:
Convert all None entries to np.NaN

### Cleaning:

In [45]:
tweets_info.replace('None',np.nan)
dogs_info.replace('None', np.nan)

,tweet_id,name,rating_numerator,rating_denominator,stage
0,892420643555336193,Phineas,13,10,NaN
1,892177421306343426,Tilly,13,10,NaN
2,891815181378084864,Archie,12,10,NaN
3,891689557279858688,Darla,13,10,NaN
4,891327558926688256,Franklin,12,10,NaN
5,891087950875897856,NaN,13,10,NaN
6,890971913173991426,Jax,13,10,NaN
7,890729181411237888,NaN,13,10,NaN
8,890609185150312448,Zoey,13,10,NaN
9,890240255349198849,Cassie,14,10,doggo


### Testing:

In [46]:
tweets_info.info()
dogs_info.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 2345 entries, 0 to 2344
Data columns (total 14 columns):
tweet_id                      2345 non-null object
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2345 non-null datetime64[ns]
source                        2345 non-null object
text                          2345 non-null object
retweeted_status_id           170 non-null float64
retweeted_status_user_id      170 non-null float64
retweeted_status_timestamp    170 non-null datetime64[ns]
expanded_urls                 2286 non-null object
retweet_count                 2345 non-null int64
favorite_count                2345 non-null int64
rate_numerator                1 non-null float64
rate_denominator              1 non-null float64
dtypes: datetime64[ns](2), float64(6), int64(2), object(4)
memory usage: 354.8+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (t

# Save the cleaned datasets.
There are three datasets to be saved.
1. tweets_info - This dataset contains information about the tweets.
2. dogs_info - This dataset contains information about dogs extracted from tweets.
3. tweets_prediction - This dataset contains dog breed prediction from dog images from tweets. This can be clubbed with dogs_info but I'm keeping it separte since it contains very specific information.


## Save datasets

In [47]:
tweets_info.to_csv("twitter_archive_master.csv")

In [48]:
dogs_info.to_csv("twitter_dogs_info.csv")

In [49]:
tweets_prediction_clean.to_csv("twitter_dog_breed_prediction.csv")